In [1]:
import pandas as pd
import os
import numpy as np
from scipy.ndimage import gaussian_filter1d
from typing import Tuple, List
import glob

# Get an interval of segmentation data (Overlay2)

This part convert segmentation data from an interval in a movie into the format that can be used by Overlay2.

In [2]:
def get_frequency_ground_truth(second_boundaries, second_interval=1,
                            end_second=555) -> Tuple:
    frequency, bins = np.histogram(second_boundaries,
                                bins=np.arange(0, end_second + second_interval,
                                                second_interval))
    return frequency, bins

def get_point_biserial(boundaries_binned, binned_comp, scale=True) -> float:
    M_1 = np.mean(binned_comp[boundaries_binned != 0])
    M_0 = np.mean(binned_comp[boundaries_binned == 0])

    n_1 = np.sum(boundaries_binned != 0)
    n_0 = np.sum(boundaries_binned == 0)
    n = n_1 + n_0

    s = np.std(binned_comp)
    r_pb = (M_1 - M_0) / s * np.sqrt(n_1 * n_0 / (float(n) ** 2))
    if scale:
        num_boundaries = boundaries_binned.astype(bool).sum()
        fake_upper = np.zeros(np.shape(binned_comp), dtype=bool)
        fake_upper[np.argsort(binned_comp)[-num_boundaries:]] = True
        M_1 = np.mean(binned_comp[fake_upper != 0])
        M_0 = np.mean(binned_comp[fake_upper == 0])
        r_upper = (M_1 - M_0) / s * np.sqrt(n_1 * n_0 / (float(n) ** 2))

        fake_lower = np.zeros(np.shape(binned_comp), dtype=bool)
        fake_lower[np.argsort(binned_comp)[:num_boundaries]] = True
        M_1 = np.mean(binned_comp[fake_lower != 0])
        M_0 = np.mean(binned_comp[fake_lower == 0])
        r_lower = (M_1 - M_0) / s * np.sqrt(n_1 * n_0 / (float(n) ** 2))
        return (r_pb - r_lower) / (r_upper - r_lower)
    else:
        return r_pb
class SegmentationVideo:
    """
    This class receives a data_frame and a video_path to extract segmentation results
    for that video
    """

    def __init__(self, data_frame, video_path):
        video_path = video_path.replace('kinect', 'C1').replace('C2', 'C1')
        if 'trim' not in video_path:
            video_path = video_path + '_trim'
        if 'C1' not in video_path:
            video_path = video_path + '_C1'
        self.data_frame = data_frame[
            data_frame['Movie'] == os.path.splitext(os.path.basename(video_path))[0]]
        self.n_participants = 0
        self.biserials = None
        self.seg_points = None
        self.gt_freqs = None
        self.gt_boundaries = None

    @staticmethod
    def string_to_segments(raw_string: str) -> np.ndarray:
        """
        This method turn a raw string to a list of timepoints
        :param raw_string: a string in csv file
        :return: list_of_segments: a list of timepoints
        """
        raw_string = raw_string.split('\n')
        list_of_segments = [float(x.split(' ')[1]) for x in raw_string if 'BreakPoint' in x]
        list_of_segments = np.array(list_of_segments)
        return list_of_segments

    
    def get_gt_freqs(self, second_interval=1, end_second=555):
        all_seg_points = np.hstack(self.seg_points)
        self.gt_boundaries, _ = get_frequency_ground_truth(all_seg_points,
                                                           second_interval=second_interval,
                                                           end_second=end_second)
        self.gt_freqs = self.gt_boundaries / self.n_participants
        self.gt_freqs = gaussian_filter1d(self.gt_freqs, 2)

        return self.gt_freqs

    def get_biserial_subjects(self, second_interval=1, end_second=555, number_boundary_range=(1, 555)):
        self.biserials = []
        if self.gt_freqs is None:
            self.get_gt_freqs(second_interval=second_interval, end_second=end_second)

        for seg_point in self.seg_points:
            participant_seg, _ = get_frequency_ground_truth(seg_point,
                                                            second_interval=second_interval,
                                                            end_second=end_second)
            # if sum(participant_seg) == 0:
            #     logger.info(f'Subject has no segments within end_second={end_second}')
            #     continue
            if sum(participant_seg) < number_boundary_range[0] or sum(participant_seg) > number_boundary_range[1]:
                print(f'Subject has {sum(participant_seg)} boundaries, outside of {number_boundary_range}')
                continue

            point = get_point_biserial(participant_seg.astype(bool), self.gt_freqs)
            if not np.isnan(point):  # some participants yield null bicorr
                self.biserials.append(point)
        return self.biserials

    def preprocess_segments(self, second_interval=1):
        # Preprocess individual segmentation to avoid undue influence
        new_seg_points = []
        for seg in self.seg_points:
            if len(seg) > 0:
                new_seg = np.array([])
                # For an interval, each participant have at most one vote
                for i in range(0, round(min(max(seg), 1500)) + 1, second_interval):
                    # seg is a list of timepoints, averaging all timepoints within a second
                    if seg[(seg > i) & (seg < i + 1)].shape[0]:
                        new_seg = np.hstack([new_seg, seg[(seg > i) & (seg < i + 1)].mean()])
            else:
                new_seg = seg
            new_seg_points.append(new_seg)
        self.seg_points = new_seg_points

        average = np.mean([len(seg) for seg in self.seg_points if len(seg)])
        std = np.std([len(seg) for seg in self.seg_points if len(seg)])
        empty = 0
        out_lier = 0
        new_seg_points = []
        for seg in self.seg_points:
            if len(seg) > 0:
                if average + 2 * std > len(seg) > average - 2 * std:
                    new_seg_points.append(seg)
                else:
                    out_lier += 1
            else:
                empty += 1
        assert len(self.seg_points) == empty + out_lier + len(new_seg_points)
        # logger.info(f'{empty} Empty participants and {out_lier} Outlier participants')
        self.seg_points = new_seg_points
        self.n_participants = len(self.seg_points)

    def get_human_segments(self, n_annotators=100, condition='coarse', second_interval=1) -> List:
        """
        This method extract a list of segmentations, each according to an annotator
        :param n_annotators: number of annotators to return
        :param condition: coarse or fine grains
        :param second_interval: interval to bin segmentations
        :return:
        """
        seg = self.data_frame[self.data_frame['Condition'] == condition]
        # parse annotations, from string to a list of breakpoints for each annotation
        annotators = list(set(seg.workerId))
        # logger.info(f'Total of participants {len(annotators)}')
        seg_points = []
        for a in annotators:
            tmp = seg[seg.workerId == a].Sec.to_numpy()
            seg_points.append(tmp)
        # seg_processed = seg['segment1'].apply(SegmentationVideo.string_to_segments)
        self.seg_points = seg_points[:n_annotators]
        self.preprocess_segments(second_interval=second_interval)
        return self.seg_points

In [5]:
seg_data = pd.read_csv('./seg_data_analysis_clean.csv')
movie = '3.3.4_C1_trim.mp4'
start = 345
end = 385
seg_video = SegmentationVideo(data_frame=seg_data, video_path=movie)
seg_video.get_human_segments(n_annotators=100, condition='fine', second_interval=1)
fine_boundaries = [x[(x > start) * (x < end)] for x in seg_video.seg_points]
fine_boundaries = np.concatenate(fine_boundaries)
# this function aggregate subject boundaries, apply a gaussian kernel and calculate correlations for subjects
gt_fine = seg_video.get_gt_freqs(second_interval=1, end_second=500)

seg_video = SegmentationVideo(data_frame=seg_data, video_path=movie)
seg_video.get_human_segments(n_annotators=100, condition='coarse', second_interval=1)
coarse_boundaries = [x[(x > start) * (x < end)] for x in seg_video.seg_points]
coarse_boundaries = np.concatenate(coarse_boundaries)
# this function aggregate subject boundaries, apply a gaussian kernel and calculate correlations for subjects
gt_coarse = seg_video.get_gt_freqs(second_interval=1, end_second=500)

In [10]:
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1, subplot_titles=("Fine", "Coarse"))
fig.add_trace(px.line(gt_fine[345:385], title='fine').data[0], row=1, col=1)
fig.add_trace(px.line(gt_coarse[345:385], title='coarse').data[0], row=2, col=1)
fig.show()

In [9]:
coarse_boundaries = coarse_boundaries - start
fine_boundaries = fine_boundaries - start
with open(f'3.3.4_C1_{start}_{end}_cut.txt', 'wt') as f:
    f.write('Coarse ' + ' '.join(coarse_boundaries.astype(str)) + '\n')
    f.write('Fine ' + ' '.join(fine_boundaries.astype(str)) + '\n')

# Get convoluted features (Overlay1)

This part convert visual statistics (part of data quality checking for fMRI) into the format that can be used by Overlay1.

In [16]:
feature_dir = r"C:\Users\nguye\Box\DCL_ARCHIVE\Documents\Events\exp152_fMRIneuralmechanisms\voxelwiseAnalyses\knitr_tan\median_correlation_with_shifts\out_convo_visual"

runs = ['1.2.3', '2.4.1', '3.1.3', '6.3.9']
features = ['luminance_mean', 'luminance_var', 'pixel_change_mean', 'pixel_change_var']
for r in runs:
    df = pd.DataFrame()
    for f in features:
        csv_file = glob.glob(os.path.join(feature_dir, f'*{r}_{f}*.csv'))[0]
        tmp = pd.read_csv(csv_file)
        df[f'{f}'] = tmp.columns.astype(float)
    df['time'] = np.arange(0, 1000, 1.483)[:len(df)]
    df['frame'] = (df['time'] * 30).astype(int)
    df.to_csv(f'./{r}_{f}_convo_overlay1.csv', index=False)
df

,luminance_mean,luminance_var,pixel_change_mean,pixel_change_var,time,frame
0,0.000000,0.000000,0.000000,0.000000,0.000,0
1,0.000278,-0.006234,0.011316,-0.007583,1.483,44
2,0.000276,-0.117490,0.122457,-0.077297,2.966,88
3,-0.037238,-0.432881,0.249790,-0.137768,4.449,133
4,-0.103133,-0.817382,0.220980,-0.185270,5.932,177
...,...,...,...,...,...,...
453,0.022442,-0.316261,0.085054,0.168055,671.799,20153
454,-0.045021,-0.290946,0.166591,0.256266,673.282,20198
455,-0.146079,-0.338427,0.225959,0.307477,674.765,20242
456,-0.210616,-0.479267,0.213587,0.270605,676.248,20287
